## Recurrent Neural Networks 

- Learn about embedding layer
- Introduce Recurrent Neural Networks (RNNs)
- Implement RNN for sentiment analysis
- Implement Long-Short Term Memories (LSTMs) for sentiment analysis
- Implement Gated Recurrent Units (GRUs) for sentiment analysis

### General
- This notebook was last tested on Python 3.8, PyTorch 1.7.1 and TorchText 0.8.1
- This notebook uses torchtext to process datasets

### Getting Started

In [1]:
# required imports
import pandas as pd
import torchtext
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import TabularDataset
import torch
import torch.nn as nn

#### Embedding Layer

Embedding layer is the ubiquitous input layer of deep neural networks used in NLP.

The [``Embedding`` layer](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#embedding) in Pytorch (that we used in the last week's tutorial on Word2vec) is a lookup table that is typically (in NLP) used to store word embeddings of a fixed vocabulary and word embedding size. Word embeddings can be retrieved from the lookup table by providing a list of word index as input to the layer. We need to know what the input and the output of this layer look like. Let's first look at a dummy example where we have two sentences ``x_1`` and ``x_2``. Let's assume we have the two sentences as:

In [2]:
x_1 = "He is very nice" # sentence 1
x_2 = "She is very kind" # sentence 2

Let's convert the two sentences into indexes (each word is replaced with its index in the vocabulary).
Let's assume our ``vocabulary size`` is set to 100. Remember, vocabulary size is a hyper-parameter.(It's because we can actually change this by putting limitations on the least frequency of words)


Let's also store that ``vocabulary size`` in a variable ``VOCAB_SIZE`` now as we will need to pass it to the ``Embedding`` layer later.

In [3]:
x_1 = [1, 25, 40, 5]
x_2 = [4, 25, 40, 99]
VOCAB_SIZE = 100

#### Max sequence length

One last thing we need to think about is the ``length of each sequence``. The two examples above are nicely set to equal length = 4. This does not need to be the case, as we can have sequences of varying lengths. We will be passing a batch of sentences to Pytorch and the max sequence length will be set to the length of the longest sentence (after tokenization) in that batch. The rest of sentences (shorter ones) will be padded with zeros. 

Now, do we need to explicitly provide the max sequence length to Pytorch? And how do we know the max seq length for each batch, if different batches have sequences of varying lengths and each batch is set to the max sent in that batch? Well, rest assured, we don't really need to worry about that. Pytorch will assign a max seq length for each batch. We will be able to inspect the max seq length for a given batch using output of the ``Embedding`` layer. (We will see that soon).

#### Size of word vector

The ``Embedding`` layer will give us a vector for each word in the vocabulary.

Now, we will need to tell it what size we want for that vector. Popular values for a vector size are usually between 100-300 for many tasks (e.g., sentiment analysis). Let's set it to 300 dimensions. 

All words in the vocabulary will have the same embedding size. Let's put that hyper-parameter in a variable ``WORD_VEC_SIZE``:

In [4]:
WORD_VEC_SIZE= 300 # size of word embedding

We are now ready to call the ``Embedding`` class to construct an embeddings tensor:

In [5]:
# Constructing an embedding Layer:
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE)
print("size of the embedding lookup table = ", embedding.weight.data.size())

# let's create a sample input (word indices) to the embedding layer 
sample_input = torch.LongTensor([ x_1, x_2 ])
print("input (word indices) tensor = \n", sample_input)
print("input (word indices) shape = ", sample_input.size())

size of the embedding lookup table =  torch.Size([100, 300])
input (word indices) tensor = 
 tensor([[ 1, 25, 40,  5],
        [ 4, 25, 40, 99]])
input (word indices) shape =  torch.Size([2, 4])


Let's pass the input to the embedding layer and print the word embeddings:

In [6]:
# let's pass the input to the embedding layer
word_embeddings = embedding(sample_input)
print("word embeddings tensor = \n", word_embeddings)

word embeddings tensor = 
 tensor([[[ 0.1901, -2.3038,  1.6044,  ...,  1.3484, -0.5816, -0.3130],
         [-0.5178, -1.4132,  1.3756,  ..., -0.9488, -0.4345,  0.9516],
         [-0.2492, -1.2017,  1.7837,  ...,  1.2783,  1.6616,  0.4356],
         [ 0.6053, -1.2433, -0.1071,  ...,  1.0977,  0.8681, -0.0269]],

        [[-0.3506, -0.1355,  0.9561,  ...,  1.1970,  1.2279,  1.1410],
         [-0.5178, -1.4132,  1.3756,  ..., -0.9488, -0.4345,  0.9516],
         [-0.2492, -1.2017,  1.7837,  ...,  1.2783,  1.6616,  0.4356],
         [ 0.6506,  0.7046,  1.2624,  ...,  1.7854,  0.6201,  1.2388]]],
       grad_fn=<EmbeddingBackward0>)


> Notice here each word in one sentence forms an embedding of (1,300) vector. Since we have 4 sentences in each document, the word_embedding size becomes (2,4,300).

Let's print the shape of this tensor:

In [7]:
print("word embeddings shape = ", word_embeddings.size())

word embeddings shape =  torch.Size([2, 4, 300])


Each dimension can be interpreted as:
- **First dimension:** (**2**,4,300): We have ``2 examples`` (that is, our ``x_1`` and ``x_2``). (Note: We will be passing a whole batch to the ``Embedding`` class and so this first dimension will be equal to the ``batch size``.
- **Second dimension:** (2,**4**,300): For each of the two examples, we have a ``max sequence length`` = 4 (x_1 and x_2 each had 4 indexes).
- **Third dimension:** (2,4,**300**): The ``word vector dimension`` is set to 300.

### Max sequence length: Another note

Recall from above we mentioned Pytorch automatically infers the max sequence length for each batch. 
For the example above (as you can see from the second dimension returned by ``word_embeddings.size()``, Pytorch 
inferred the max seq length for this batch of two sentences is 4.

Let's just adjust the second example, **adding two more words** (the string "and kind"). Note, both our ``VOCAB_SIZE`` and ``WORD_VEC_SIZE`` stay the same as before. We assign the word "and" an index of "7" and the word "considerate" an index of "60". 

Note that we have to pad the first example ``x_1`` with zeros (we will explicitly set the padding index to zero later when we define the embedding layer) in the end. (Try removing the zero padding. You will get an error.):

In [8]:
x_1 = "He is very nice"
x_2 = "She is very kind and considerate"
x_1 = [1, 25, 40, 5, 0, 0]   # the zeros must be manually padded here
x_2 = [4, 25, 40, 99, 7, 60]

Now, let's create a new ``embedding`` layer by creating a new instance of the ``Embedding`` class:

In [9]:
# constructing an embedding layer:
padded_embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE, padding_idx=0)
print("size of the embedding lookup table = ", padded_embedding.weight.data.size())

size of the embedding lookup table =  torch.Size([100, 300])


Note that **padded_embedding** embedding layer specifies the padding index (corresponds to an embedding initialized to all zeros). In this example, the zeroth index is dedicated for storing padding embedding (vector initialized to all zeros).

Let's create sample input and pass it to the embedding layer.

In [10]:
# let's create a sample input
sample_input = torch.LongTensor([ x_1, x_2 ])
print("input (word indices) tensor = \n", sample_input)
print("input (word indices) shape = ", sample_input.size())

# let's retrieve the word embeddings by passing the sample input to the layer
word_embeddings = padded_embedding(sample_input)
print(word_embeddings)

input (word indices) tensor = 
 tensor([[ 1, 25, 40,  5,  0,  0],
        [ 4, 25, 40, 99,  7, 60]])
input (word indices) shape =  torch.Size([2, 6])
tensor([[[-0.6576,  1.5036, -1.1379,  ..., -0.7042, -0.3216,  0.6035],
         [ 2.0925, -0.2163,  1.0738,  ..., -1.3819, -1.5071, -2.0384],
         [-0.3410,  0.9473,  0.2688,  ...,  0.7882,  1.2361,  0.7345],
         [-0.8911,  0.1360, -0.1262,  ...,  0.1120,  0.4622, -1.0694],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 2.0051,  0.1782,  1.1753,  ...,  0.1490, -0.0251, -0.7231],
         [ 2.0925, -0.2163,  1.0738,  ..., -1.3819, -1.5071, -2.0384],
         [-0.3410,  0.9473,  0.2688,  ...,  0.7882,  1.2361,  0.7345],
         [ 1.5075, -0.4779, -0.3377,  ...,  0.9493,  1.3547, -1.0004],
         [ 0.5206, -0.6316,  0.4947,  ...,  0.5175, -0.6837,  0.3200],
         [-0.1060, -2.1154, -1.3870,  ..., -0.9027, -0.4275,  0.680

If we inspect the shape of the new tensor ``word_embeddings``, we will see the second dimension now changed to 6, to match the max sequence length:

In [11]:
print(word_embeddings.size())

torch.Size([2, 6, 300])


### How does Pytorch initialize word vector dimensions/weights?

Note that Pytorch initializes the word vectors from a **normal distribution** $ \mathcal{N}(0, 1) $. The word embedding weights are by default learnable parameters in Pytorch and so they will be adjusted during training. (Note: These weights can be initialized from an external word embedding tool such as [Word2vec](https://code.google.com/archive/p/word2vec/), [Fasttext](https://fasttext.cc/), or [Glove](https://nlp.stanford.edu/projects/glove/)). Also, the weights can be frozen (by setting ``embedding.weigh.required_grad`` flag to False), which is a reasonable option when initialized from an external tool. You can choose to keep learning them within the model with your training data). Below we show the ones initialized from a normal distribution by Pytorch.

In [12]:
embedding.weight

Parameter containing:
tensor([[ 0.7115,  1.1786, -0.3746,  ...,  0.4107,  1.3485,  0.3292],
        [ 0.1901, -2.3038,  1.6044,  ...,  1.3484, -0.5816, -0.3130],
        [ 0.7274, -0.2286,  1.8600,  ...,  0.1848,  2.1144,  0.3943],
        ...,
        [ 1.2256, -0.1416, -0.5852,  ..., -0.4734, -1.0416, -0.2759],
        [ 0.7502, -0.0846, -2.0633,  ..., -0.6296, -0.6940,  0.2222],
        [ 0.6506,  0.7046,  1.2624,  ...,  1.7854,  0.6201,  1.2388]],
       requires_grad=True)

More information about the ``Embedding`` class can be found [here](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) ([source code](https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html#Embedding)).

## Recurrent Neural Networks
Recurrent Neural Networks (RNNs) are used to model sequences of arbitrary length (e.g., sequence of words in a sentence, sequence of sentences in a document, sequence of frames in a video). RNNs typically use their internal state (memory) to process sequence of inputs. 

At each time-step, RNNs output a prediction and hidden state, feeding its previous hidden state into each next step. RNNs are applied in a wide range of NLP applications:
- language modeling, where RNN can condition on **all** previous words in the corpus unlike n-gram language model
- text classification, where the states act as features (we will see sentiment analysis in this notebook)
- machine translation, where a RNN is used to process a sentence in source language and another RNN is used to decode the sentence in target language (we will see this in the "Machine Translation" folder)
- sequence labeling, where the states in RNN are used to predict a category for each item in the sequence 

Recommended reading for understanding the theory of RNNs: https://github.com/UBC-NLP/dlnlp2019/blob/master/slides/RNN.pdf 


### Grabbing few tweets using torchtext

Let us follow [**torchtext** tutorial](https://github.com/Georgeanna-Li/Machine_Learning/blob/master/Supervised_ml/Pytorch/torchtext_tutorial.ipynb) to read few tweets from the [sentiment analysis dataset](http://alt.qcri.org/semeval2016/task4/) used in the previous tutorial on feedforward neural networks. The preprocessed (tokenization, removing URLs, mentions, hashtags and so on) tweets are placed under ``data`` folder in three files as ``train.tsv``, ``dev.tsv`` and ``test.tsv``.  

Let us view few tweets from ``train.tsv`` using pandas.

In [13]:
import pandas as pd
df = pd.read_csv("./data/sentiment_analysis/train.tsv", sep = '\t', header=None, names=['tweet','label']) # the separator of tsv file is `\t`
df.head()

,tweet,label
0,dear <<<MENTION>>> the newooffice for mac is g...,2
1,<<<MENTION>>> how about you make a system that...,2
2,i may be ignorant on this issue but should we ...,2
3,thanks to <<<MENTION>>> i just may be switchin...,2
4,if i make a game as a <<<HASHTAG>>> universal ...,0


**We import the relevant packages, define the tokenizer and TorchText's fields.**

In [14]:
# import related packages
import torchtext
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import TabularDataset

# define the white space tokenizer to get tokens
def tokenize_en(tweet):
    """
    Tokenizes English tweet from a string into a list of strings (tokens)
    """
    return tweet.strip().split()

# define the TorchText's fields
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)

**To use the different splits (training, development and testing), we use `TabularDataset` class to load datasets.**

In [15]:
train, val, test = TabularDataset.splits(
    path="./data/sentiment_analysis/", # the root directory where the data lies
    train='train.tsv', validation="dev.tsv", test="test.tsv", # file names
    format='tsv',
    skip_header=False, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
    fields=[('tweet', TEXT), ('label', LABEL)])

**Build our vocabulary to map words to integers.**

In [16]:
TEXT.build_vocab(train, min_freq=3) # builds vocabulary based on all the words that occur at least twice in the training set
LABEL.build_vocab(train)

---------------------

**Notice that this part is a false example of what will happen if we set `sort` as `False`.**

**Initialize the iterators for the train, validation, and test data. Note that we set ``sort`` as `False` so as to not sort examples based on similar lengths which minimizes padding.**

In [17]:
from torchtext.legacy.data import Iterator, BucketIterator

train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(4,64,64),
 sort_key=lambda x: len(x.tweet), 
 sort=False,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=False
)

**Create a batch of four examples and print them**

In [18]:
# create a single batch and terminate the loop
for batch in train_iter:
    tweets = batch.tweet
    labels = batch.label
    break  #we use first batch as an example.

# print the four examples with padding and corresponding label
print("processed tweets: ")
for j in range(tweets.shape[1]): # sample loop
    tokens = []
    for i in range(tweets.shape[0]): # token loop
        tokens.append(TEXT.vocab.itos[tweets[i,j]])
    print(j," sample:",tokens," label:", labels[j].item())

processed tweets: 
0  sample: ['the', 'only', 'thing', 'that', 'i', 'am', 'afraid', 'of', 'if', 'bernie', 'sanders', 'is', 'president', 'he', 'may', 'die', 'in', 'office', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']  label: 1
1  sample: ['iphone', 'launch', 'more', 'details', 'leak', 'ahead', 'of', "apple's", 'september', '<<<digit>>>', 'event', 'iphone', 'launch', 'on', 'wednesday', 'is', 'just', '<<<url>>>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']  label: 1
2  sample: ['finishing', 'jurassic', 'park', 'for', 'like', 'the', 'time', 'tomorrow', 'film', 'class', 'is', 'gonna', 'have', 'a', 'lot', 'of', '<unk>', 'for', 'me', 'all', 'the', 'batman', 'films', 'star', 'wars']  label: 1
3  sample: ['in', 'the', '<unk>', 'of', 'being', 'fail', 'and', 'balanced', 'may', 'i', 'present', 'the', '<<<mention>>>', 'galaxy', 'note', '<<<digit>>>', '<<<url>>>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']  label: 1


**Notice that the above part is not a good example where we set `sort` as `False`!**

We see that there are a lot of paddings in those processed tweets. The reason is that we are not sorting sentences by length, therefore we put some sentences with different lengths together, which results in unnecessary paddings.

-----------------

**Now we set ``sort`` as `True` so as to sort examples based on similar lengths which minimizes padding.**

**Let us initialize the new iterators for the train, validation, and test data.**

In [19]:
train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(4,64,64),
 sort_key=lambda x: len(x.tweet), 
 sort=True,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=True
)

**Let us pick up 4 tweets from the training set and convert them to tensors.**

**Create a batch of four examples and print them**

In [20]:
# create a single batch and terminate the loop
for batch in train_iter:
    tweets = batch.tweet
    labels = batch.label
    print(tweets)
    print(labels)
    break  #we use first batch as an example.

tensor([[ 191,    4, 1273,   49],
        [  14,   82,    2,    0],
        [   2,   73,  215,   18],
        [ 598,  145,   48,  215],
        [  21,   21,   21,   21]])
tensor([0, 1, 0, 0])


In [21]:
# print the four examples with padding and corresponding label
print("processed tweets: ")
for j in range(tweets.shape[1]): # sample loop
    tokens = []
    for i in range(tweets.shape[0]): # token loop
        tokens.append(TEXT.vocab.itos[tweets[i,j]])     # itos = index to string
    print(j," sample:",tokens," label:", labels[j].item())

processed tweets: 
0  sample: ['ihop', 'is', 'the', 'move', 'tomorrow']  label: 0
1  sample: ['<<<mention>>>', 'make', 'david', 'beckham', 'tomorrow']  label: 1
2  sample: ['bringing', 'the', 'bentley', 'out', 'tomorrow']  label: 0
3  sample: ['new', '<unk>', 'with', 'bentley', 'tomorrow']  label: 0


In [22]:
tweets

tensor([[ 191,    4, 1273,   49],
        [  14,   82,    2,    0],
        [   2,   73,  215,   18],
        [ 598,  145,   48,  215],
        [  21,   21,   21,   21]])

<br>

<br>

### Creating a single hidden layer RNN

PyTorch has ``torch.nn.RNN`` module that implements the vanilla (Elman) RNN with *tanh* or *ReLU* non-linearity. The documentation for this module is [here](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html?highlight=nn%20rnn#torch.nn.RNN). Let us use the sample batch of five examples created before to understand this module.

Here we will represent the input tweet using a sequence of word embeddings (for each word present in the tweet). We will use ``torch.nn.Embedding`` layer to store word vectors corresponding to words in the vocabulary.

Before implementing the embedding module for our usecase, let us compute the size of the word vocabulary.

In [23]:
# print the size of the word vocabulary
VOCAB_SIZE = len(TEXT.vocab.stoi)
print(VOCAB_SIZE)

3333


We have 3333 unique words in the vocabulary.

Let us implement the embedding module (whose underlying weight matrix shape is (``vocabulary size`` $\times$ ``word embedding size``) for our usecase:

In [24]:
# set the word embedding size
WORD_VEC_SIZE = 300

# an Embedding module containing 300 dimensional tensor for each word in the vocabulary
# Note, the parameters to Embedding class below are:
# num_embeddings (int): size of the dictionary of embeddings
# embedding_dim (int): the size of each embedding vector
# For more details on Embedding class, see: https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/sparse.py
embedding = nn.Embedding(VOCAB_SIZE, WORD_VEC_SIZE, sparse=True)
print("lookup table shape = ", embedding.weight.size())

lookup table shape =  torch.Size([3333, 300])


Let us now feed the tensors of our sample batch to the embedding module and extract the sequence of word embeddings for each tweet.

In [25]:
# print tensor containing word ids for our batch
print("*"*50, "\n Word ids for the first batch (recall, it has 4 sentences, each column representing a sentence): \n", tweets.data, "\n","*"*50,)

# feed the "word ids" tensor to the embedding module
tweet_input_embeddings = embedding(tweets)

# print the dimensions of the tweet_embeddings
print("*"*50, "\n Tweet input word embeddings size: ", tweet_input_embeddings.size(), "\n","*"*50,) 
# first dimension - sequence length: number of words per example (same across the whole batch, after padding) --> max_seq = 22
# second dimension -  batch size / number of examples in the batch --> 4
# third dimension - number of dimensions in the word vector

************************************************** 
 Word ids for the first batch (recall, it has 4 sentences, each column representing a sentence): 
 tensor([[ 191,    4, 1273,   49],
        [  14,   82,    2,    0],
        [   2,   73,  215,   18],
        [ 598,  145,   48,  215],
        [  21,   21,   21,   21]]) 
 **************************************************
************************************************** 
 Tweet input word embeddings size:  torch.Size([5, 4, 300]) 
 **************************************************


Here the dimensions are not typical, because normally in a neural network we would have the first dimension as *batch size*, the second dimension as *sequence length*, and the third dimension as *number of dimensions in the word vector*. 

So we should always be careful about the dimensions.


<br>

<br>

Let's actually view the actual word embeddings tensor for this batch:

- Take the first matrix for example, the first line is the *first token of the tweet 1*, and the second line is the *first token of the tweet 2*,...

- For the second matrix, the first line is the *second token of the tweet 1*...

In [26]:
print("*"*50, "\n Embeddings for the first batch: \n", tweet_input_embeddings, "\n","*"*50,) 

************************************************** 
 Embeddings for the first batch: 
 tensor([[[ 0.0497, -0.6613,  0.8796,  ..., -0.4001,  1.3097, -0.4884],
         [ 0.1035,  0.8628, -1.6050,  ..., -0.2178,  1.2235, -0.4696],
         [ 0.2006,  0.0309,  0.6056,  ...,  0.2487,  0.3876,  1.3356],
         [-1.2187,  0.3973, -0.8658,  ...,  1.3622,  0.8258,  0.1237]],

        [[-0.2521,  0.4899,  0.2510,  ..., -0.2158,  0.9611, -1.4206],
         [ 0.9853,  1.4709,  0.7310,  ..., -0.7995,  0.6693, -1.1515],
         [ 0.2335, -0.0525,  0.1288,  ...,  0.4827,  0.1625,  1.3136],
         [ 2.0688,  1.0594, -1.3368,  ...,  2.0239,  1.2129,  0.7414]],

        [[ 0.2335, -0.0525,  0.1288,  ...,  0.4827,  0.1625,  1.3136],
         [-1.7450, -0.9481,  1.0052,  ..., -0.1025, -1.1641, -1.5934],
         [ 0.9274,  0.7615, -0.4993,  ..., -0.4135,  0.5053,  1.0365],
         [ 1.0659, -0.4157, -0.1426,  ..., -0.3867,  1.8647,  0.0849]],

        [[-0.0097, -0.2096, -0.0502,  ...,  1.7240, -1.

What we are seeing is the actual word vectors representing each of the 4 sentences (i.e., whole batch).
This is dimension 2 in ``tweet_input_embeddings``. 

In [27]:
tweet_input_embeddings.size()[1]

4

As mentioned, ``max_seq length`` for this batch is ``5``, which is dimension 1 (indexed as 0 in Pytorch, similar to Python) 
in ``tweet_input_embeddings``: 

In [28]:
tweet_input_embeddings.size()[0]

5

Now, dimension 3 in ``tweet_input_embeddings`` (indexed as 2) is the size of the word vectors:

In [29]:
tweet_input_embeddings.size()[2]

300

Let's look at the vector for the ``first word`` in the ``first sentence`` in the batch:

In [30]:
tweet_input_embeddings[:1, :1, :].shape

torch.Size([1, 1, 300])

In [31]:
tweet_input_embeddings[:1, :1, :]

tensor([[[ 0.0497, -0.6613,  0.8796, -2.1471, -0.9065, -0.1080, -1.8510,
           0.8044,  1.2097, -0.7341,  0.8311,  0.7651, -1.1590,  0.5495,
           0.6600, -0.5957, -0.9349,  0.1479,  0.5176, -1.7738,  0.1265,
          -1.3657, -0.1073, -2.2124, -0.6851,  0.9176,  0.2406,  0.5285,
           0.4733, -1.1413,  0.3945,  0.6287,  0.0211,  0.4979, -1.3710,
          -0.1173,  1.0671, -0.3403,  0.9572,  1.2945,  1.2748, -0.5902,
           0.8075,  2.3248,  0.4592, -0.2317, -1.5478,  0.5803, -0.9309,
           1.1187, -0.0721,  0.1367,  0.1587, -0.2785,  0.6543, -0.8597,
          -0.9292,  0.9744,  0.7614, -0.1196, -1.2134,  0.6489, -0.4959,
           1.9132,  0.1632,  0.5426,  1.2953,  1.6276,  1.5345,  2.0170,
          -1.5114, -0.2768,  1.7806,  0.9310,  0.9351, -0.0390, -0.2544,
           0.5106,  2.1638,  0.4476, -0.7431,  0.5342, -0.5459, -0.2583,
           0.3176, -1.4729,  1.0111,  0.4005,  0.1107, -0.5714,  1.3365,
           0.3088, -0.9442,  1.5314,  1.3513,  0.84

Let's look at the ``first 5 dimensions`` of that same ``first word`` of the ``first sentence``:

In [32]:
tweet_input_embeddings[:1, :1, :5]

tensor([[[ 0.0497, -0.6613,  0.8796, -2.1471, -0.9065]]],
       grad_fn=<SliceBackward0>)

The following shows you the ``first 5 dimensions`` of the ``first word`` from ``each of the 4 sentences``

In [33]:
tweet_input_embeddings[:1, :, :5]

tensor([[[ 0.0497, -0.6613,  0.8796, -2.1471, -0.9065],
         [ 0.1035,  0.8628, -1.6050, -1.6371,  0.6035],
         [ 0.2006,  0.0309,  0.6056, -0.3022,  0.0536],
         [-1.2187,  0.3973, -0.8658, -0.2913,  1.1139]]],
       grad_fn=<SliceBackward0>)

The following shows you the ``last 7 dimensions`` of the ``last word`` from ``the last sentence``.

In [34]:
tweet_input_embeddings[-1:, -1:, -7:]

tensor([[[ 1.3923, -0.4266, -0.0244,  0.4538, -0.0131, -0.0657, -0.1492]]],
       grad_fn=<SliceBackward0>)

<br>
<br>

We will be passing the sequence of word embeddings for each sentence in the batch as input to the RNN. But let's now define an RNN module first:

In [35]:
"""
define the RNN module
"""
# first input - number of dimensions for word vectors for a vector x (300, size of the word embedding)
# second input - number of nodes in hidden state h_t (50, size of the hidden layer)
# third input - number of recurrent layers (we set it to 1)
rnn = nn.RNN(input_size=300, hidden_size=50, num_layers=1) # input_size, hidden_size, num_layers
print(rnn)

RNN(300, 50)


<br>
<br>

We will now pass the ``tweet_input_embeddings`` (representations of words in our batch) to RNN. Before we do, we need to know RNN also *optionally* takes a parameter for the ``initial hidden state h0`` (that is, the hidden state we will input to the model before the forward propagation starts. If this vector is not explicitly specified, Pytorch will just initialize h0 to a tensor of zeros.)

Let's construct an ``initial hidden state h0``. Pay attention to the shape of its tensor, and what each of the 3 parameters 
mean.

In [36]:
"""
hidden layer at time-step 0 (h_0)
"""
# first dimension - number of RNN layers (1)
# second dimension - number of examples/sentences in a batch
# third dimension - number of nodes in hidden layer (50, size of the hidden layer, that we specified as hidden_size in RNN construction)
h0 = torch.randn(1, 4, 50)
print("The shape as as expected: ", h0.shape)

The shape as as expected:  torch.Size([1, 4, 50])


<br>

Let us feed both the hidden representation constructed above and tweet embeddings to our RNN model.
We will get back two objects ``output`` and ``hn`` that we will need to understand.

In [37]:
"""
forward propagation over the RNN model
"""
output, hn = rnn(tweet_input_embeddings, h0) 
# h0 is optional input, defaults to tensor of 0's of apprpriate size (num_layers, batch, hidden_size) when not provided

But what is ``output``? Well, let's inspect its shape first:

In [38]:
# output = seq_len, batch, hidden_size (output features from last layer of RNN)
print("output size: ", output.size())

output size:  torch.Size([5, 4, 50])


Here's what we need to know about ``output``:
- The first dimension in the ``output`` tensor is the ``max_seq length`` (5). 
- The second dimension is ``batch_size`` (the number of examples/sentences in our batch = 4).
- The third dimension is the ``size of nodes/units`` in our hidden layer (=50). 

What is the shape of `hn` (tensor containing the hidden state for t=max_seq_length) ?

In [39]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([1, 4, 50])


Here's what we need to know about ``hn``:
- ``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of hidden layer nodes) containing the hidden state for the last ``time step`` 
(``t = max_seq_length``).

You can take the output representation for a tweet after processing the last token (t=seq_len or last timestep) and call the resulting representation as the tweet representation that **"summarizes" the information present** in the tweet. This tweet representation can further be used for a useful task like tweet classification (we will try out sentiment analysis later in this tutorial) by adding a classification module on top of the tweet representation.

> This `h_n` is the recurrent hidden layer of RNN.

Let us compute the final tweet representation:

In [40]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


In [41]:
tweet_output_embeddings

tensor([[ 0.1722,  0.9785, -0.4916,  0.8167,  0.9822, -0.8901,  0.7265,  0.9476,
          0.6588,  0.0333,  0.6143, -0.7835, -0.3580, -0.4351,  0.7930,  0.1546,
         -0.7270,  0.9206,  0.9844, -0.7004,  0.6254, -0.2912, -0.8590, -0.5337,
          0.9171,  0.7269,  0.8216,  0.2037,  0.8335,  0.8913,  0.2106, -0.2162,
         -0.8760,  0.9680, -0.9720, -0.7904, -0.1621, -0.4894, -0.9454, -0.9038,
          0.8667, -0.8328,  0.3629, -0.3457,  0.7354, -0.6668,  0.5052, -0.9672,
          0.4521,  0.4315],
        [ 0.2722,  0.9633,  0.3640,  0.4598,  0.9462, -0.9340,  0.6756,  0.8687,
          0.2608,  0.6020,  0.5295, -0.6816,  0.0441,  0.1948,  0.9107, -0.6932,
         -0.9248,  0.9949,  0.9446, -0.8027,  0.3860, -0.1418, -0.5424, -0.9260,
          0.8161,  0.8401,  0.9568,  0.4214,  0.6950,  0.9749, -0.3559,  0.3205,
         -0.9784,  0.9413, -0.8600, -0.9490,  0.1046, -0.4076, -0.9377, -0.8248,
          0.6900, -0.7823, -0.1246, -0.3968,  0.8810, -0.9490, -0.4256, -0.9937,


<br>

<br>

<br>

## Multilayered RNN

For some applications, we may need more than one hidden layer for RNN to model the information flow. Adding more layers only requires few changes.

Firstly, we change the ``num_layers`` argument to reflect the number of layers we want during the RNN module definition (we will define two hidden layers).

In [42]:
"""
define the RNN module
"""
# first input - number of dimesnions for word vectors for a vector x (300, size of the word embedding)
# second input - number of nodes in hidden layer (50, size of the hidden layer)
# third input - number of recurrent layers (we set it to 2)
rnn = nn.RNN(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Similar to single layered RNN, Multilayered RNN module takes two inputs: the ``initial hidden state h0`` for each element in the batch (at ``time step t=0``) and the ``input features`` (``tweet_input_embeddings`` in our case).

Let us construct the new initial hidden state for a 2 layered RNN.

In [43]:
"""
hidden layer at time-step 0 (h_0)
"""
# first dimension - number of RNN layers (2)
# second dimension - number of examples/sentences in a batch (4)
# third dimension - number of nodes in hidden layer (50, size of the hidden layer)
h0 = torch.randn(2, 4, 50)
print("The shape as as expected: ", h0.shape)

The shape as as expected:  torch.Size([2, 4, 50])


Let us feed both the hidden representation constructed above and tweet embeddings to our RNN model.

In [44]:
"""
forward propagation over the RNN model
"""
print(tweet_input_embeddings.shape)
output, hn = rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's when not provided

torch.Size([5, 4, 300])


``output`` tensor contains the output features $h_t$ from the last layer of the RNN

In [45]:
# output = seq_len, batch, hidden_size (output features from last layer of RNN)
print("output size: ", output.size())

output size:  torch.Size([5, 4, 50])


``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of nodes in a hidden layer) containing the hidden state for last time step ``t = max_seq_len`` for the ``2 layered RNN``.

In [46]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


#### Building tweet representation

Actually, `output` is tensor containing the output features (h_t) from the last layer of the RNN, for each t. Namely, `output` returns all the hidden states of all time steps from the last layer of the RNN. Hence, the last element of `output` is `h_n`. 
Let us print them out:

In [47]:
print("last element of output:\n", output[-1])

last element of output:
 tensor([[-2.3755e-01,  2.0197e-01, -4.5273e-01,  1.9134e-01, -6.4322e-02,
          5.7595e-01,  6.2657e-01,  8.2071e-02, -1.2430e-03, -7.1888e-01,
         -2.1794e-01, -2.0135e-01,  3.5560e-02,  7.9951e-02,  4.5778e-01,
          1.0291e-01,  3.5427e-01, -3.7307e-02,  6.1022e-01,  6.7984e-02,
          3.8143e-01,  2.4028e-01,  6.5714e-01,  5.3988e-01, -8.7714e-02,
          4.6788e-01, -4.8817e-01, -6.7583e-01, -3.3352e-02, -1.0845e-01,
          5.9589e-02,  5.0715e-01,  5.4126e-01,  1.7227e-02, -5.2711e-02,
          7.4660e-01,  4.2723e-01,  4.2414e-01, -4.2217e-01,  2.3839e-01,
          1.7724e-03,  7.6602e-02,  2.5165e-01, -7.8974e-02,  7.2263e-01,
          3.4349e-01, -3.0204e-01, -7.6930e-01,  2.5551e-01,  5.3589e-01],
        [-3.3300e-01,  4.5231e-01, -4.6670e-01,  1.1326e-02,  6.5857e-02,
          6.2729e-01,  5.3091e-01,  6.5794e-01, -9.8252e-02, -6.1484e-01,
         -2.4441e-01,  1.0896e-01,  2.3490e-01,  7.0867e-01,  4.7210e-01,
         -2.

Then let us print out the last hidden state of last layer. 
Notice, we have two RNN layer, we only want to use the hidden state of last layer. 
You can find the these value are same as `output[-1]`.

In [48]:
print("last hidden state h_n:\n", hn[-1])

last hidden state h_n:
 tensor([[-2.3755e-01,  2.0197e-01, -4.5273e-01,  1.9134e-01, -6.4322e-02,
          5.7595e-01,  6.2657e-01,  8.2071e-02, -1.2430e-03, -7.1888e-01,
         -2.1794e-01, -2.0135e-01,  3.5560e-02,  7.9951e-02,  4.5778e-01,
          1.0291e-01,  3.5427e-01, -3.7307e-02,  6.1022e-01,  6.7984e-02,
          3.8143e-01,  2.4028e-01,  6.5714e-01,  5.3988e-01, -8.7714e-02,
          4.6788e-01, -4.8817e-01, -6.7583e-01, -3.3352e-02, -1.0845e-01,
          5.9589e-02,  5.0715e-01,  5.4126e-01,  1.7227e-02, -5.2711e-02,
          7.4660e-01,  4.2723e-01,  4.2414e-01, -4.2217e-01,  2.3839e-01,
          1.7724e-03,  7.6602e-02,  2.5165e-01, -7.8974e-02,  7.2263e-01,
          3.4349e-01, -3.0204e-01, -7.6930e-01,  2.5551e-01,  5.3589e-01],
        [-3.3300e-01,  4.5231e-01, -4.6670e-01,  1.1326e-02,  6.5857e-02,
          6.2729e-01,  5.3091e-01,  6.5794e-01, -9.8252e-02, -6.1484e-01,
         -2.4441e-01,  1.0896e-01,  2.3490e-01,  7.0867e-01,  4.7210e-01,
         -2.3

> Notice that the last element of output is exactly the same as the last timestep of hidden layer.

Let us compute the final tweet representation:

In [49]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


In [50]:
tweet_output_embeddings

tensor([[-2.3755e-01,  2.0197e-01, -4.5273e-01,  1.9134e-01, -6.4322e-02,
          5.7595e-01,  6.2657e-01,  8.2071e-02, -1.2430e-03, -7.1888e-01,
         -2.1794e-01, -2.0135e-01,  3.5560e-02,  7.9951e-02,  4.5778e-01,
          1.0291e-01,  3.5427e-01, -3.7307e-02,  6.1022e-01,  6.7984e-02,
          3.8143e-01,  2.4028e-01,  6.5714e-01,  5.3988e-01, -8.7714e-02,
          4.6788e-01, -4.8817e-01, -6.7583e-01, -3.3352e-02, -1.0845e-01,
          5.9589e-02,  5.0715e-01,  5.4126e-01,  1.7227e-02, -5.2711e-02,
          7.4660e-01,  4.2723e-01,  4.2414e-01, -4.2217e-01,  2.3839e-01,
          1.7724e-03,  7.6602e-02,  2.5165e-01, -7.8974e-02,  7.2263e-01,
          3.4349e-01, -3.0204e-01, -7.6930e-01,  2.5551e-01,  5.3589e-01],
        [-3.3300e-01,  4.5231e-01, -4.6670e-01,  1.1326e-02,  6.5857e-02,
          6.2729e-01,  5.3091e-01,  6.5794e-01, -9.8252e-02, -6.1484e-01,
         -2.4441e-01,  1.0896e-01,  2.3490e-01,  7.0867e-01,  4.7210e-01,
         -2.3280e-01,  1.2244e-01, -4

## RNN for Sentiment Analysis

In this section we will implement RNN for classifying the sentiment of the tweet (same task used in our previous feedforward neural networks tutorial).

We will pick up most of the functions from our feedforward neural networks code:

In [51]:
# all the necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim

# set the seed (for reproducibility)
manual_seed = 123
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
  torch.cuda.manual_seed(manual_seed)

# hyperparameters
MAX_EPOCHS = 5 # number of passes over the training data
LEARNING_RATE = 0.3 # learning rate for the weight update rule
NUM_CLASSES = 3 # number of classes for the problem
EMBEDDING_SIZE = 300 # size of the word embedding

Now we can define the full RNN model:

> For the dimension in `nn.LogSoftmax(dim=1)`, we are doing normalization over the classes(the second dimension).

In [72]:
"""
create a model for RNN
"""
class RNNmodel(nn.Module):
  
  def __init__(self, embedding_size, vocab_size, output_size, hidden_size, num_layers):
    # In the constructor we define the layers for our model
    super(RNNmodel, self).__init__()
    # word embedding lookup table
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size, sparse=True)
    # core RNN module
    self.rnn_layer = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers) 
    # activation function
    self.activation_fn = nn.ReLU()
    # classification related modules
    self.linear_layer = nn.Linear(hidden_size, output_size) 
    self.softmax_layer = nn.LogSoftmax(dim=1)
    self.debug = False
  
  def forward(self, x):
    # In the forward function we define the forward propagation logic
    if self.debug:
        print("input word indices shape = ", x.size())
    out = self.embedding(x)
    if self.debug:
        print("word embeddings shape = ", out.size())
    out, _ = self.rnn_layer(out) # since we are not feeding h_0 explicitly, h_0 will be initialized to zeros by default
    if self.debug:
        print("RNN output (features from last layer of RNN for all timesteps) shape = ", out.size())
    # classify based on the hidden representation after RNN processes the last token
    out = out[-1]   # get the final output
    if self.debug:
        print("Tweet embeddings or RNN output (features from last layer of RNN for the last timestep only) shape = ", out.size())
    out = self.activation_fn(out)
    if self.debug:
        print("ReLU output shape = ", out.size())
    out = self.linear_layer(out)
    if self.debug:
        print("linear layer output shape = ", out.size())
    out = self.softmax_layer(out) # accepts 2D or more dimensional inputs
    if self.debug:
        print("softmax layer output shape = ", out.size())
    return out

Some additional hyperparameters for RNN

In [53]:
# hyperparameters of RNN
HIDDEN_SIZE = 50 # no. of units in the hidden layer
NUM_LAYERS = 2 # no. of hidden layers

Rest of the pipeline looks similar to our feedforward neural networks code (except that we are using **torchtext** instead of **DataLoader**):

In [54]:
from sklearn.metrics import accuracy_score

def train(loader):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.tweet
        batch_output = batch.label
    
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)

        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        total_loss += cur_loss.item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]
    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.tweet
            batch_output = batch.label

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    return accuracy

Let us define the RNN model.

In [55]:
# define the model
model = RNNmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS) 
model.to(device) # ship it to the right device

# define the loss function (last node of the graph)
criterion = nn.NLLLoss()

Let us make a full forward propagation pass over a sample input batch to the RNN model. Closely pay attention to the shapes of intermediate layers (by turning on debug mode of the model)

In [56]:
# turn on the debug mode
model.debug = True

# print the sample input batch and labels
print("sample input = ", tweets)
print("sample output = ", labels)

# feed the batch as input to the RNN model
model_prediction = model(tweets)
print('model prediction shape = ', model_prediction.size())

# feed the model prediction and labels to the loss function
loss = criterion(model_prediction, labels)
print("loss = ", loss.item())

# turn off the debug mode (as we go for training from now)
model.debug = False

sample input =  tensor([[ 191,    4, 1273,   49],
        [  14,   82,    2,    0],
        [   2,   73,  215,   18],
        [ 598,  145,   48,  215],
        [  21,   21,   21,   21]])
sample output =  tensor([0, 1, 0, 0])
input word indices shape =  torch.Size([5, 4])
word embeddings shape =  torch.Size([5, 4, 300])
RNN output (features from last layer of RNN for all timesteps) shape =  torch.Size([5, 4, 50])
Tweet embeddings or RNN output (features from last layer of RNN for the last timestep only) shape =  torch.Size([4, 50])
ReLU output shape =  torch.Size([4, 50])
linear layer output shape =  torch.Size([4, 3])
softmax layer output shape =  torch.Size([4, 3])
model prediction shape =  torch.Size([4, 3])
loss =  1.115674614906311


**We need to create a new directory 'ckpt/' to store our model checkpoint.**

In [57]:
import os
if not os.path.exists("./ckpt"): # check if the directory doesn't exist already
    os.mkdir("./ckpt")

**Let us perform the training. We will save our model and optimizer at end of each epoch.**


You can find more information of saving and loading model [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

In [58]:
# create an instance of SGD with required hyperparameters
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

# start the training
for epoch in range(MAX_EPOCHS):
    # train the model for one pass over the data
    train_loss = train(train_iter)  
    # compute the training accuracy
    train_acc = evaluate(train_iter)
    # compute the validation accuracy
    val_acc = evaluate(val_iter)
    
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_acc, val_acc))
    
    # save model, optimizer, and number of epoch to a dictionary
    model_save = {
            'epoch': epoch,  # number of epoch
            'model_state_dict': model.state_dict(), # model parameters 
            'optimizer_state_dict': optimizer.state_dict(), # save optimizer 
            'loss': train_loss # training loss
            }
    
    # use torch.save to store 
    torch.save(model_save, "./ckpt/model_{}.pt".format(epoch))

Epoch [1/5], Loss: 0.2506, Training Accuracy: 0.5157, Validation Accuracy: 0.4217
Epoch [2/5], Loss: 0.2480, Training Accuracy: 0.5157, Validation Accuracy: 0.4217
Epoch [3/5], Loss: 0.2483, Training Accuracy: 0.5157, Validation Accuracy: 0.4217
Epoch [4/5], Loss: 0.2495, Training Accuracy: 0.5153, Validation Accuracy: 0.4217
Epoch [5/5], Loss: 0.2504, Training Accuracy: 0.5170, Validation Accuracy: 0.4217


We trained the network only for 5 epochs, but it already overfits on validation set after epoch 2. 
In the coming sessions, we will look at methods to ``regularize`` the network (this will help us deal with overfitting).

**Load model checkpoint** 

When we have a trained model checkpint, we can load it using `torch.load()`

In [59]:
# define a new model
model2 = RNNmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS) 

# load checkpoint 
checkpoint = torch.load("./ckpt/model_1.pt") # loading the model obatined after 2nd epoch

# assign the parameters of checkpoint to this new model
model2.load_state_dict(checkpoint['model_state_dict'])
model2.to(device)

print(model2) # can be used for inference or for further training

RNNmodel(
  (embedding): Embedding(3333, 300, sparse=True)
  (rnn_layer): RNN(300, 50, num_layers=2)
  (activation_fn): ReLU()
  (linear_layer): Linear(in_features=50, out_features=3, bias=True)
  (softmax_layer): LogSoftmax(dim=1)
)


## GRUs

Gated Recurrent Units (GRUs) are a variant of RNNs that use more complex units for activation. They are created to have more persistent memory thereby making them easier for RNNs to capture long-term dependencies.

GRU is defined by ``torch.nn.GRU`` module and its documentation can be fetched [here](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#gru). Now let us define the GRU module.

In [60]:
"""
define the GRU module
"""
# first input - number of word vector dimensions/embeddings
# second input - number of nodes in hidden layer (50, size of the hidden layer)
# third input - number of recurrent layers (2)
gru_rnn = nn.GRU(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Similar to RNN, GRU module takes two inputs: *the initial hidden state for each element in the batch* (t=0) and the *input features* (``tweet_input_embeddings`` in our case).

Let us feed both the initial hidden state and tweet embeddings to our GRU model.

In [61]:
"""
forward propagation over the GRU model
"""
output, hn = gru_rnn(tweet_input_embeddings, h0) # h0 is optional input, defaults to tensor of 0's when not provided

``output`` tensor contains the output features $h_t$ from the last layer of the GRU

In [62]:
# output = seq_len, batch, hidden_size (output features from last layer of GRU)
print("output size: ", output.size())

output size:  torch.Size([5, 4, 50])


``hn`` is a tensor of shape (num_layers, batch_size, hidden_size / number of nodes in a hidden layer) containing the hidden state for last time step ``t = max_seq_len`` for the ``2 layered RNN``.

In [63]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


Similar to RNN, you can compute the final tweet representation (representation from last hidden state for each tweet) as follows.

In [64]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (5)
# second dimension - number of features in hidden state h_t (20, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


## LSTMs

Long short-term memory (LSTMs) are a variant of RNNs that use more complex units for activation. Similar to the spirit of GRU, they are created to have more persistent memory thereby making them easier for RNNs to capture long-term dependencies.

LSTM is defined by ``torch.nn.LSTM`` module and its documentation can be fetched [here](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#lstm). Now let us define the LSTM module.

In [65]:
"""
define the LSTM module
"""
# first input - number of features in x (300, size of the word embedding)
# second input - number of number of nodes in a hidden layer (50)
# third input - number of recurrent layers (2)
lstm_rnn = nn.LSTM(input_size=300, hidden_size=50, num_layers=2) # input_size, hidden_size, num_layers

Unlike RNN and GRU, LSTM module takes **three inputs**: the **initial hidden state** for each element in the batch (t=0), the **input features** (tweet_input_embeddings in our case) and the **initial cell state** for each element in the batch.

Let us construct the initial cell state (this construction is similar to that of initial hidden state)
> This cell state is somewhat related to how much a cell has to forget.

In [66]:
"""
cell state at time-step 0 (h_0)
"""
# first dimension - number of LSTM layers (2)
# second dimension - batch_size (# of tweets/examples/sentences)
# third dimension - hidden_size / number of nodes in a hidden layer (50)
c0 = torch.randn(2, 4, 50)

Let us feed the initial hidden state, initial cell state and tweet embeddings to our LSTM model.

In [67]:
"""
forward propagation over the LSTM model
"""
output, (hn, cn) = lstm_rnn(tweet_input_embeddings, (h0, c0)) # h0 and c0 is optional input, defaults to tensor of 0's when not provided

``output`` tensor contains the output features $h_t$ from the last layer of the LSTM

In [68]:
# output = seq_len, batch_size, hidden_size (output features from last layer of LSTM)
print("output size: ", output.size())

output size:  torch.Size([5, 4, 50])


``hn`` is a tensor of shape (num_layers, batch, hidden_size) containing the hidden state for t = seq_len

In [69]:
# h_n = num_layers, batch, hidden_size (hidden state for t=seq_len or hidden state at last timestep)
print("last hidden state size: ", hn.size())

last hidden state size:  torch.Size([2, 4, 50])


``cn`` is a tensor of shape (num_layers, batch, hidden_size) containing the cell state for t = seq_len.

In [70]:
# c_n = num_layers, batch_size, hidden_size (cell state for t=seq_len or cell state at last timestep)
print("last cell state size: ", cn.size())

last cell state size:  torch.Size([2, 4, 50])


Similar to RNN and GRU, you can compute the final tweet representation (representation from last hidden state for each tweet) as follows.

In [71]:
tweet_output_embeddings = output[-1,:,:] # -1 fetches the embeddings from the last timestep
print("tweet output embeddings size: ", tweet_output_embeddings.size())
# first dimension - number of tweets in the batch (4)
# second dimension - number of features in hidden state h_t (50, size of the hidden layer)

tweet output embeddings size:  torch.Size([4, 50])


That's it!